In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
import pandas as pd
import PIL
import tifffile
import re

from skimage import measure
from skimage.io import imread
from skimage.segmentation import clear_border

import ipywidgets as widgets

sys.path.append('../libraries')
from ring_functions import make_rings
import input_functions as inp_f

In [2]:
info_file_path = r'Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\livecell_B03\full\data\info_B03.txt'

In [3]:
# read the info file
info_file = open(info_file_path, 'r')
info_lines = info_file.readlines()
info_file.close()

# read info about the data frame
exp_dir,df_name = inp_f.read_df_info(info_lines)

# get info about the channels
channel_list = inp_f.read_channels(info_lines)
print(channel_list)

# setting directories
labels_dir = os.path.join(exp_dir,'segmentation')
im_dir = os.path.join(exp_dir,'data')
df_dir = os.path.join(exp_dir,'df')

# reading labels 
file_list = [x for x in os.listdir(labels_dir) if 'label' in x]

[{'channel_name': 'PCNA ', 'file_name': 'WellB03_ChannelCFP_Seq0001 - Stitched.ome.tif', 'channel_in_file': 0, 'channel_number': 0, 'color': 'yellow', 'tracking': 1}]


In [4]:
# setting properties to calculate
properties = ['label', 'area','centroid','orientation','major_axis_length','minor_axis_length','bbox','image','mean_intensity','image_intensity']
properties_ring = ['label','centroid','mean_intensity']

In [5]:
# create slider
progress_bar = widgets.IntProgress(
    step=1,
    description='Processing:',
    orientation='horizontal',
    min = 0,
    max = len(file_list)-1,
    value = 0
)

Use this version to pull channel number from file name post downscale and rejoin scripts

In [6]:
#%%time

display(progress_bar)
cellDataList=[]

PIL.Image.MAX_IMAGE_PIXELS = None

cellDataList = []

for label_file in file_list:
    frame = int(label_file.split('_')[-2])
    print(frame)
    
    # Assuming progress_bar is a widget or a mechanism to display progress
    progress_bar.value = frame
    print(progress_bar.value)
    
    # Open labels
    label_path = os.path.join(labels_dir, label_file)
    print(label_path)
    labels_2D = imread(label_path)
    print(labels_2D)
    
    # Clear border objects
    labels_2D = clear_border(labels_2D)
    
    # Read images for intensity calculations
    intensity_list = []
    for file in os.listdir(os.path.join(exp_dir, 'data')):
        print(file)
        # Extract the channel number from the file name - Can only be done if preprocessed using downscaling and merging scripts
        match = re.search(r'channel_(\d+)_output', file)
        print(match)
        if match:
            channel_number = int(match.group(1))
            im_path = os.path.join(exp_dir, 'data', file)
            print(im_path)
            
            # Open the image; assuming c=0 because each movie has only one channel and t=frame for the timepoint
            im = inp_f.open_image(im_path, c=0, t=frame)  # Adjust this line according to how images are opened in your context

            mean_intensity = np.mean(im)
            print(f'Mean intensity of channel {channel_number}: {mean_intensity}')
            
            
            intensity_list.append(im)
    
    # Assuming the order in intensity_list corresponds to the correct channel order
    int_im = np.moveaxis(np.array(intensity_list), 0, 2)
    
    # Calculate properties of regions
    cellData = pd.DataFrame(measure.regionprops_table(labels_2D, properties=properties, intensity_image=int_im))
    
    # Add info of these measurements
    cellData['file'] = label_file
    cellData['t'] = frame
    
    # Calculate signals in rings
    rings = make_rings(labels_2D, width=6, gap=1)
    rings_prop = measure.regionprops_table(rings, properties=properties_ring, intensity_image=int_im)
    rings_prop = pd.DataFrame(rings_prop)
    
    cellData = pd.merge(cellData, rings_prop, how='inner', on='label', suffixes=('_nuc', '_ring'))

    cellData.rename(columns={'centroid-0_nuc': 'centroid-0', 'centroid-1_nuc': 'centroid-1'}, inplace=True)
    
    cellDataList.append(cellData)

# Put all together
cellDataAll = pd.concat(cellDataList, ignore_index=True)

# Add info regarding image size
cellDataAll['size_x'] = labels_2D.shape[0]
cellDataAll['size_y'] = labels_2D.shape[1]

# Save calculations
cellDataAll.to_pickle(os.path.join(df_dir, df_name), protocol=4)
cellDataAll.to_csv(os.path.join(df_dir, df_name.replace('pkl', 'csv')), index=False)

print("Processing complete.")

IntProgress(value=0, description='Processing:', max=187)

0
0
Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\livecell_B03\full\segmentation\WellB03_ChannelCFP_Seq0001 - Stitched.ome_000_label.png
<PIL.TiffImagePlugin.TiffImageFile image mode=I;16 size=13427x13432 at 0x1A87B563508>


TypeError: int() argument must be a string, a bytes-like object or a number, not 'TiffImageFile'

Use this version to pull channel number from txt file

In [7]:
#%%time

# Display the progress bar or any other mechanism if applicable
display(progress_bar)

cellDataList = []

PIL.Image.MAX_IMAGE_PIXELS = None

# Example channel list containing mapping info (update this list as needed)
channel_list = [
    {'channel_name': 'PCNA', 'file_name': 'WellB03_ChannelCFP_Seq0001 - Stitched.ome.tif',
     'channel_in_file': 0, 'channel_number': 0, 'color': 'yellow', 'tracking': 1}
    # Add more channel information as necessary
]

for label_file in file_list:
    # Extract the frame number from the file name
    frame = int(label_file.split('_')[-2])
    print(f"Processing frame {frame}")
    
    # Update the progress bar value
    progress_bar.value = frame
    print(progress_bar.value)
    
    # Open the label file
    label_path = os.path.join(labels_dir, label_file)
    print(f"Label file path: {label_path}")
    labels_2D = imread(label_path)
    print(labels_2D)
    
    # Clear objects touching the border
    labels_2D = clear_border(labels_2D)
    
    # Initialize an empty list to collect the intensity images
    intensity_list = []

    # Iterate over the channel list to process the corresponding images
    for channel_info in channel_list:
        file_name = channel_info['file_name']
        channel_index = channel_info['channel_in_file']
        
        # Construct the path to the current file
        im_path = os.path.join(exp_dir, 'data', file_name)
        print(f"Processing {file_name} at channel index {channel_index}")

        # Open the image using the specified channel index
        im = inp_f.open_image(im_path, c=channel_index, t=frame)
        
        # Add the image to the intensity list
        intensity_list.append(im)
    
    # Ensure that the intensity list contains data
    if intensity_list:
        # Arrange data to (height, width, channels)
        int_im = np.moveaxis(np.array(intensity_list), 0, 2)
    else:
        print("No intensity images found for the current frame.")
        continue
    
    # Calculate region properties with intensity information
    cellData = pd.DataFrame(measure.regionprops_table(labels_2D, properties=properties, intensity_image=int_im))
    
    # Add additional information to the dataframe
    cellData['file'] = label_file
    cellData['t'] = frame
    
    # Calculate signals in rings
    rings = make_rings(labels_2D, width=6, gap=1)
    rings_prop = measure.regionprops_table(rings, properties=properties_ring, intensity_image=int_im)
    rings_prop = pd.DataFrame(rings_prop)
    
    # Merge the ring properties with the main cell data
    cellData = pd.merge(cellData, rings_prop, how='inner', on='label', suffixes=('_nuc', '_ring'))
    
    # Rename centroid columns to avoid conflicts
    cellData.rename(columns={'centroid-0_nuc': 'centroid-0', 'centroid-1_nuc': 'centroid-1'}, inplace=True)
    
    # Append the current cell data to the main list
    cellDataList.append(cellData)

# Concatenate all the dataframes
cellDataAll = pd.concat(cellDataList, ignore_index=True)

# Add information regarding image size
cellDataAll['size_x'] = labels_2D.shape[0]
cellDataAll['size_y'] = labels_2D.shape[1]

# Save the final calculations
cellDataAll.to_pickle(os.path.join(df_dir, df_name), protocol=4)
cellDataAll.to_csv(os.path.join(df_dir, df_name.replace('pkl', 'csv')), index=False)

print("Processing complete.")


IntProgress(value=0, description='Processing:', max=187)

Processing frame 0
0
Label file path: Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\livecell_B03\full\segmentation\WellB03_ChannelCFP_Seq0001 - Stitched.ome_000_label.png
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Processing WellB03_ChannelCFP_Seq0001 - Stitched.ome.tif at channel index 0
Processing frame 1
1
Label file path: Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\livecell_B03\full\segmentation\WellB03_ChannelCFP_Seq0001 - Stitched.ome_001_label.png
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
Processing WellB03_ChannelCFP_Seq0001 - Stitched.ome.tif at channel index 0
Processing frame 2
2
Label file path: Z:\Garrett\Livecell\072723_50hr_20uM_TBHP\livecell_B03\full\segmentation\WellB03_ChannelCFP_Seq0001 - Stitched.ome_002_label.png
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0